In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def calc_eps_max(x):
    # FIXED: n,p,p_max,k
    # VARIABLE: tau,psi,e1,e2,e3
    n,p,p_max,k,tau,psi,e1,e2,e3 = x
    
    # Define known quantities
    delta = 1-tau
    p_frac = (2*p-1)/(2*p-2)
    e4 = ((1-p_frac+psi-e3)**(-1))*(0.5-p_frac+psi-e3)-p
    Phi = ((1/k)-e2)*(((2*p-1)/(2*p-2))-psi-e1)
    
    # Define exponents
    exp1 = -2*(1-p_frac+psi-e3)*delta*(e4**2)*n
    exp2 = -(2*(delta**2)*(e3**2)*n)/(p_frac - psi)
    exp3 = -2*(p_frac-psi-e1)*tau*(e2**2)*n
    exp4 = -(2*(tau**2)*(e1**2)*n)/(p_frac - psi)
    
    # Calculate important quantities
    eps_ver = max(np.exp(exp1)+np.exp(exp2), np.exp(exp3)+np.exp(exp4))
    eps_rej = np.exp(-2*((Phi-p_max)**2)*tau*n)
    eps_max = eps_ver + eps_rej

    return eps_max

In [3]:
# Define constraints with input
# x = [n,p,p_max,k,tau,psi,e1,e2,e3]
#      0,1,2     3,4,   5,  6, 7, 8

# Fixed quantities
def n_cons(x):
    return x[0] - 6818
def p_cons(x):
    return x[1] - 0
def p_max_cons(x):
    return x[2] - 0.15
def k_cons(x):
    return x[3] - 2
def tau_cons(x):
    return x[4] - 0.9

# Bounds

# 0 < psi
def cons1(x):
    return x[5]
# psi < (2p-1)/(2p-2)
def cons2(x):
    return (2*x[1]-1)/(2*x[1]-2) - x[5]
# 0 < e1
def cons3(x):
    return x[6]
# e1 < 0.5 - psi
def cons4(x):
    return 0.5-x[5]-x[6]
# 0 < e2
def cons5(x):
    return x[7]
# e2 < (1/k)
def cons6(x):
    return (1/x[3])-x[7]
# 0 < e3
def cons7(x):
    return x[8]
# e3 < psi
def cons8(x):
    return x[5]-x[8]
# 0 < tau
def cons9(x):
    return x[4]
# tau < 1
def cons10(x):
    return 1-x[4]
# p_max < threshold
def cons11(x):
    return ((1/2)-x[7])*(0.5-x[5]-x[6]) - x[2]
# epsilon < 1
def cons12(x):
    return 1-calc_eps_max(x)

eqcon1 = {'type': 'eq', 'fun': n_cons}
eqcon2 = {'type': 'eq', 'fun': p_cons}
eqcon3 = {'type': 'eq', 'fun': p_max_cons}
eqcon4 = {'type': 'eq', 'fun': k_cons}
eqcon5 = {'type': 'eq', 'fun': tau_cons}
con1 = {'type': 'ineq', 'fun': cons1}
con2 = {'type': 'ineq', 'fun': cons2}
con3 = {'type': 'ineq', 'fun': cons3}
con4 = {'type': 'ineq', 'fun': cons4}
con5 = {'type': 'ineq', 'fun': cons5}
con6 = {'type': 'ineq', 'fun': cons6}
con7 = {'type': 'ineq', 'fun': cons7}
con8 = {'type': 'ineq', 'fun': cons8}
con9 = {'type': 'ineq', 'fun': cons9}
con10 = {'type': 'ineq', 'fun': cons10}
con11 = {'type': 'ineq', 'fun': cons11}
con12 = {'type': 'ineq', 'fun': cons12}

# cons = [eqcon1, eqcon2, eqcon3, eqcon4, con1, con2, con3, 
#         con4, con5, con6, con7, con8, con9, con10, con11, con12]

cons = [eqcon1, eqcon2, eqcon3, eqcon4, eqcon5, con1, con2, con3, 
        con4, con5, con6, con7, con8, con9, con10, con11, con12]

In [4]:
# x = [n,p,p_max,k,tau,psi,e1,e2,e3]
x = [6818,0,0.15,2,0.9,0.15,0.01,0.01,0.1]

test = minimize(calc_eps_max,x,constraints=cons)
print(test)

y = test['x']
print('--------------------')
print('n = ' + str(y[0]))
print('p = ' + str(y[1]))
print('p_max = ' + str(y[2]))
print('k = ' + str(y[3]))
print('tau = ' + str(y[4]))
print('psi = ' + str(y[5]))
print('e1 = ' + str(y[6]))
print('e2 = ' + str(y[7]))
print('e3 = ' + str(y[8]))
print('Phi = ' + str(((1/y[3])-y[7])*(0.5-y[5]-y[6])))
print('eps = ' + str(calc_eps_max(y)))

     fun: 0.0791770342219737
     jac: array([-4.09726053e-05,  1.37822035e+00,  5.34253331e+00,  4.79081783e-01,
        3.51088763e+00, -2.96689131e+00,  2.51854920e+00,  1.91632875e+00,
        2.38799709e+00])
 message: 'Optimization terminated successfully'
    nfev: 151
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([6.81800000e+03, 0.00000000e+00, 1.50000000e-01, 2.00000000e+00,
       9.00000000e-01, 1.30950493e-01, 1.03563453e-02, 2.85837213e-02,
       9.33952760e-02])
--------------------
n = 6818.0
p = 0.0
p_max = 0.15
k = 2.0
tau = 0.9
psi = 0.13095049255084978
e1 = 0.010356345302951875
e2 = 0.0285837212576576
e3 = 0.09339527600561214
Phi = 0.16909379570928446
eps = 0.0791770342219737
